In [2]:
import os
import re
import json
import jsonlines
from langchain.schema import Document
from langchain_experimental.text_splitter import SemanticChunker
from langchain_upstage import UpstageEmbeddings
from langchain_milvus.vectorstores import Milvus
from uuid import uuid4
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

import pandas as pd
import pytz

from datetime import timedelta
from operator import itemgetter
from langchain_teddynote.retrievers import KiwiBM25Retriever
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.chains.query_constructor.base import (
  StructuredQueryOutputParser,
  get_query_constructor_prompt
)
from langchain_teddynote.evaluator import GroundednessChecker
from langchain.retrievers.self_query.milvus import MilvusTranslator
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
import warnings
from langchain_core.runnables import chain

warnings.filterwarnings('ignore')

In [3]:
embeddings = UpstageEmbeddings(
    model='solar-embedding-1-large-query',
)

In [4]:
def save_docs_to_jsonl(documents, file_path):
    with jsonlines.open(file_path, mode="w") as writer:
        for doc in documents:
            writer.write(doc.dict())

In [5]:
def adjust_time_filter_to_week(time_filter):
    """
    특정 날짜(YYYY-MM-DD)가 주어진 경우, 해당 날짜를 포함하는 주(월~일)의
    첫 번째 날(월요일)과 마지막 날(일요일)로 변환하는 함수.

    :param time_filter: dict, {"start_date": datetime, "end_date": datetime}
    :return: dict, {"start_date": datetime, "end_date": datetime}
    """
    # Extract start_date and end_date from time_filter
    start_date = time_filter.start_date
    end_date = time_filter.end_date

    # Handle the case where start_date or end_date is None
    if start_date is None or end_date is None:
        if start_date is not None and end_date is None:
            start_of_week = start_date - timedelta(days=start_date.weekday())  # 월요일 찾기
            end_of_week = start_of_week + timedelta(days=6)  # 해당 주 일요일 찾기

            return {
                "start_date": start_of_week.replace(hour=0, minute=0, second=0),
                "end_date": end_of_week.replace(hour=23, minute=59, second=59)
            }
        elif end_date is not None and start_date is None:
            start_of_week = end_date - timedelta(days=end_date.weekday())  # 월요일 찾기
            end_of_week = start_of_week + timedelta(days=6)  # 해당 주 일요일 찾기

            return {
                "start_date": start_of_week.replace(hour=0, minute=0, second=0),
                "end_date": end_of_week.replace(hour=23, minute=59, second=59)
            }
        else:
            return None  # or return the time_filter as is if you prefer

    # 날짜가 동일한 경우, 주의 첫 번째 날(월요일)과 마지막 날(일요일)로 변경
    if start_date.year == end_date.year and start_date.month==end_date.month and start_date.day==end_date.day:
        start_of_week = start_date - timedelta(days=start_date.weekday())  # 월요일 찾기
        end_of_week = start_of_week + timedelta(days=6)  # 해당 주 일요일 찾기

        return {
            "start_date": start_of_week.replace(hour=0, minute=0, second=0),
            "end_date": end_of_week.replace(hour=23, minute=59, second=59)
        }

    # 날짜가 다르면 기존 time_filter 유지
    return {
        "start_date": start_date,
        "end_date": end_date
    }

In [6]:
from datetime import datetime
from typing import Optional
from pydantic import BaseModel
import instructor
from openai import OpenAI
from pydantic import BaseModel, Field, field_validator
from typing import Literal


class TimeFilter(BaseModel):
    start_date: Optional[datetime] = None
    end_date: Optional[datetime] = None

class SearchQuery(BaseModel):
    query: str
    time_filter: TimeFilter

class Label(BaseModel):
    chunk_id: int = Field(description="The unique identifier of the text chunk")
    chain_of_thought: str = Field(
        description="The reasoning process used to evaluate the relevance"
    )
    relevancy: int = Field(
        description="Relevancy score from 0 to 10, where 10 is most relevant",
        ge=0,
        le=10,
    )

class RerankedResults(BaseModel):
    labels: list[Label] = Field(description="List of labeled and ranked chunks")

    @field_validator("labels")
    @classmethod
    def model_validate(cls, v: list[Label]) -> list[Label]:
        return sorted(v, key=lambda x: x.relevancy, reverse=True)

def rerank_results(query: str, chunks: list[dict]) -> RerankedResults:
    client = instructor.from_openai(OpenAI())
    return client.chat.completions.create(
        model="gpt-4o-mini",
        response_model=RerankedResults,
        messages=[
            {
                "role": "system",
                "content": """
                You are an expert search result ranker. Your task is to evaluate the relevance of each text chunk to the given query and assign a relevancy score.

                For each chunk:
                1. Analyze its content in relation to the query.
                2. Provide a chain of thought explaining your reasoning.
                3. Assign a relevancy score from 0 to 10, where 10 is most relevant.

                Be objective and consistent in your evaluations.
                """,
            },
            {
                "role": "user",
                "content": """
                <query>{{ query }}</query>

                <chunks_to_rank>
                {% for chunk in chunks %}
                <chunk id="{{ chunk.id }}">
                    {{ chunk.text }}
                </chunk>
                {% endfor %}
                </chunks_to_rank>

                Please provide a RerankedResults object with a Label for each chunk.
                """,
            },
        ],
        context={"query": query, "chunks": chunks},
    )

In [7]:
def get_query_date(question):
    today = datetime(2025, 1, 25)
    days_since_last_friday = (today.weekday() - 4) % 7
    last_friday = today - timedelta(days=days_since_last_friday)
    issue_date = last_friday.strftime("%Y-%m-%d")

    client = instructor.from_openai(OpenAI())
    response = client.chat.completions.create(
        model="o1",
        response_model=SearchQuery,
        messages=[
            {
                "role": "system",
                "content": f"""
                You are an AI assistant that extracts date ranges from financial queries.
                The current report date is {issue_date}.
                Your task is to extract the relevant date or date range from the user's query
                and format it in YYYY-MM-DD format.
                If no date is specified, answer with None value.
                """,
            },
            {
                "role": "user",
                "content": question,
            },
        ],
    )

    parsed_dates = adjust_time_filter_to_week(response.time_filter)

    # parsed_dates를 순회하며 None인 경우도 처리
    if parsed_dates:
        start = parsed_dates['start_date']
        end=parsed_dates['end_date']
    else:
        start=None
        end = None

    if start is None or end is None:
        expr = None
    else:
        expr = f"issue_date >= '{start.strftime('%Y%m%d')}' AND issue_date <= '{end.strftime('%Y%m%d')}'"
        expr = expr
    return expr

In [58]:
def convert_to_list(example):
    if isinstance(example["contexts"], list):
        contexts = example["contexts"]
    else:
        try:
            contexts = json.loads(example["contexts"])
        except json.JSONDecodeError as e:
            print(f"JSON Decode Error: {example['contexts']} - {e}")
            contexts = []
    return {"contexts": contexts}

def generate_expr(question: str) -> dict:
    expr = get_query_date(question)
    return {"expr": expr}

def reranking(docs, question, k=15):
    chunks = [{"id": idx, "issue_date": doc.metadata['issue_date'],  "text": doc.page_content} for idx, doc in enumerate(docs)]
    documents_with_metadata = [{"text": doc.page_content, "metadata": doc.metadata} for doc in docs]
    reranked_results = rerank_results(query=question, chunks=chunks)

    chunk_dict = {chunk["id"]: chunk["text"] for chunk in chunks}
    top_k_results = [chunk_dict.get(label.chunk_id, "") for label in reranked_results.labels[:k] if label.chunk_id in chunk_dict]

    reranked_results_with_metadata = []
    for reranked_result in top_k_results:
        page_content = reranked_result

        matching_metadata = None
        for doc in documents_with_metadata:
            if doc["text"] == page_content:
                matching_metadata = doc["metadata"]
                break

        document = Document(
            metadata=matching_metadata,
            page_content=page_content
        )
        reranked_results_with_metadata.append(document)

    context_rerankedNbm25 = reranked_results_with_metadata
    return context_rerankedNbm25

text_prompt = PromptTemplate.from_template(
'''
Today is '2025-01-25'. 
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Answer in Korean. Answer in detail.

#Question:
{question}
#Context:
{context}

#Answer:'''
)


In [ ]:
question_answer_relevant = GroundednessChecker(
  llm=ChatOpenAI(model='o1', temperature=0), target='question-answer'
).create()

In [65]:
question = '2주 전 은행채 발행액은?'

question_answer_relevant.invoke({'question': question, 'answer': text_chain.invoke({'question': question})})


GroundnessQuestionScore(score='yes')

In [66]:
question = '2주 전 은행채 발행액은?'

question_answer_relevant.invoke({'question': question, 'answer': text_chain.invoke({'question': question})})


GroundnessQuestionScore(score='yes')

In [67]:
question = '2주 전 은행채 발행액은?'

question_answer_relevant.invoke({'question': question, 'answer': text_chain.invoke({'question': question})})


GroundnessQuestionScore(score='yes')

In [ ]:
from langchain_upstage import UpstageGroundednessCheck

# 업스테이지 Groundness Checker 생성
upstage_groundedness_check = UpstageGroundednessCheck()

In [61]:
text_chain.invoke({'question': question})

'2주 전(즉, 2025년 1월 10일 기준) 공시에 따르면 은행채 발행액은 2조 6,100억 원이었다고 명시되어 있습니다. 해당 자료에서 “전 주 대비 9,700억 원 증가”라는 언급이 있으나, 2주 전 시점인 1월 10일 공시상의 구체적인 발행액 수치는 2조 6,100억 원으로 나타납니다. 따라서 2025년 1월 25일을 기준으로 2주 전 은행채 발행액은 2조 6,100억 원입니다.'

In [62]:
result = reranking(question=question, docs=vectorstore_text.as_retriever(search_kwargs={'k': 25, 'expr':get_query_date('2주 전 은행채 발행액은?')}).invoke(question))

In [63]:
result

[Document(metadata={'page': 13, 'source': './raw_pdf_copy3\\KISWeekly제1118호(20250110).pdf', 'summary': '- 은행채 발행액: 2조 6,100억 원, 전 주 대비 9,700억 원 증가\n- 한국수출입은행 연내물 할인채: 언더 10.3bp\n- 우리은행 연내물 할인채: 언더 15.9bp\n- 국민은행 1년 만기 이표채: 언더 7.9bp\n- 농협은행 1년 만기 이표채: 언더 6.6bp\n- 기타금융채 발행액: 3조 4,600억 원, 전 주 대비 2조 8,700억 원 증가\n- A- 등급 키움캐피탈 및 엠캐피탈: 언더 20bp 이상 발행 강세\n- 엠캐피탈: 새마을금고중앙회 지주 편입으로 OUTLOOK 상향\n- A+ 등급 롯데캐피탈 및 메리츠캐피탈 강세 지속\n- AA- 등급 캐피탈채 발행 강세 지속', 'issue_date': '20250110', 'pk': 'de99baa8-ee4c-4479-b78d-ae5768509c5d', 'type': 'text'}, page_content='은행채 발행액은 2조 6,100 억 원으로 전 주 대비 9,700억원 증가했다. 은행채 발행량이 증가세를 이어간 가운데, 연내물 할인채 위주의 강세를 지속했다. 국책은행 중에서 한국수출입은행의 연내물 할인채가 언더 10.3bp로 발행되었고, 시중은행 중에서 우리은행의 연내물 할인채가 언더 15.9bp로 발행되며 강세를 이어 갔다. 이어서 이표채 중에서 역시 연내물 위주의 강세가 지속되었다. 국민은행의 1년만기 이표채가 언더 7.9bp, 농협은행의 1년만기 이표채가 언더 6.6bp로 발행되며 강세를 보였다. 이어서 한국산업은행의 1년구 간 이표채 역시 발행 강세를 보이며 마감했다. 기타금융채 발행액은 3조 4,600억원으로 전 주 대비 2조 8,700억원 증가했다.'),
 Document(metadata={'page': 12, 'source': './raw_pdf_copy3\\KISWeekly제1118호(

In [55]:
format_docs(result)

'Issue Date: 20250110\nContent: 은행채 발행액은 2조 6,100 억 원으로 전 주 대비 9,700억원 증가했다. 은행채 발행량이 증가세를 이어간 가운데, 연내물 할인채 위주의 강세를 지속했다. 국책은행 중에서 한국수출입은행의 연내물 할인채가 언더 10.3bp로 발행되었고, 시중은행 중에서 우리은행의 연내물 할인채가 언더 15.9bp로 발행되며 강세를 이어 갔다. 이어서 이표채 중에서 역시 연내물 위주의 강세가 지속되었다. 국민은행의 1년만기 이표채가 언더 7.9bp, 농협은행의 1년만기 이표채가 언더 6.6bp로 발행되며 강세를 보였다. 이어서 한국산업은행의 1년구 간 이표채 역시 발행 강세를 보이며 마감했다. 기타금융채 발행액은 3조 4,600억원으로 전 주 대비 2조 8,700억원 증가했다.\n\nIssue Date: 20250110\nContent: 은행채 발행시장은 발행량을 증가고, 연내물 위주로 큰폭의 스프레드를 축소했다. 은행채 1년 구간이 언 더 8.0bp로 지속적인 축소세를 나타냈으며, 그밖에도 3년 및 10년 구간으로도 축소세를 지속했다. 3년 구간 이 4.0bp 촉소했고, 5년 및 10년 구간이 각각 1.0bp, 1.5bp 축소세를 보이며 마감했다. 기타금융채 발행시장은 발행량을 증가 전환했다. 기타금융채 역시 연내물을 중심의 스프레드 축소세를 이어갔다. 1년 구간이 언더 7.5bp로 스프레드를 더욱 축소했고, 이어서 3년 구간은 3.0bp 축소되었다.\n\nIssue Date: 20250110\nContent: 은행채는 유통시장은 연내물 위주의 강세로 출발했다. 주 초반 연내물을 중심으로 언어 3.0bp 이상의 큰 강세로 시작한 은행채 시장은 주중으로 갈수록 강세가 확대되는 모습을 나타냈다. 특시 산금 및 중금, 은행 채 AA0까지의 전반적인 등급에서 언더 3.0bp 이상의 강세를 지속했다. 주 후반 역시 강세 폭을 줄이기는 했 지만 연내물 1년 물을 중심으로 강세를 지속하며 마감했다. 기타금융채시장 역시 유통 강세

In [9]:
question_answer_relevant = GroundednessChecker(
  llm=ChatOpenAI(model='gpt-4o-mini', temperature=0), target='question-answer'
).create()

@chain
def kill_table(result):
    if question_answer_relevant.invoke({'question': result['question'], 'answer': result['text']}).score == 'no':
        result['context'] = table_chain.invoke({'question': result['question']})
    else:
        result['context'] = result['text']
    return result

In [15]:
filepath = './chunked_jsonl/250313_text_semantic_per_80.jsonl'

splitted_doc_text = []
with open(filepath, 'r', encoding='utf-8') as file:
    for line in file:
        if line.startswith('\n('):
            continue
        data = json.loads(line)

        doc = Document(
            page_content=data['page_content'],
            metadata=data['metadata']
        )
        splitted_doc_text.append(doc)

In [16]:
# splitted_doc_text = []
# for i in splitted_doc:
#   if i.metadata['issue_date'][-2:] != '00':
#     splitted_doc_text.append(i)

In [17]:
URI = 'http://127.0.0.1:19530'

vectorstore_text = Milvus(
    embedding_function=embeddings,
    connection_args={'uri':URI},
    index_params={'index_type': 'AUTOINDEX', 'metric_type': 'IP'},
    collection_name='text_semantic_per_80_00_test'
)

# uuids = [str(uuid4()) for _ in range(len(splitted_doc_text))]

# vectorstore_text.add_documents(
#   documents=splitted_doc_text,
#   ids=uuids
# )

In [18]:
vectorstore_predict = Milvus(
    embedding_function=embeddings,
    connection_args={'uri':URI},
    index_params={'index_type': 'AUTOINDEX', 'metric_type': 'IP'},
    collection_name='text_semantic_per_80_00'
)

In [19]:

milvus_retriever_text = vectorstore_text.as_retriever(
    search_kwargs={'k':20}
)

bm25_retriever_text = KiwiBM25Retriever.from_documents(
    splitted_doc_text
)
bm25_retriever_text.k = 20


In [20]:
filepath = './chunked_jsonl/table_v7.jsonl'

splitted_doc_table = []
with open(filepath, 'r', encoding='utf-8') as file:
    for line in file:
        if line.startswith('\n('):
            continue
        data = json.loads(line)

        doc = Document(
            page_content=data['page_content'],
            metadata=data['metadata']
        )
        splitted_doc_table.append(doc)

In [21]:
embeddings=UpstageEmbeddings(
    model='solar-embedding-1-large-query'
)

vectorstore_table = Milvus(
  embedding_function=embeddings,
  connection_args={'uri':URI},
  index_params={'index_type': 'AUTOINDEX', 'metric_type': 'IP'},
  collection_name='table_v7'
)

In [22]:
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

bm25_retriever_table = KiwiBM25Retriever.from_documents(
    splitted_doc_table
)
bm25_retriever_table.k = 5

In [23]:
def format_docs(docs):
    # 각 문서의 issue_date와 page_content를 함께 출력하도록 포맷합니다.
    return "\n\n".join(
        f"Issue Date: {doc.metadata.get('issue_date', 'Unknown')}\nContent: {doc.page_content}"
        for doc in docs
    )

In [59]:
llm_text = ChatOpenAI(model='o1', temperature=1)

answer = []

text_chain = (
    RunnableParallel(
        question=itemgetter('question')
    ).assign(expr = lambda x: get_query_date(x['question'])
    ).assign(context_raw=lambda x: RunnableLambda(
            lambda _: vectorstore_text.as_retriever(
                search_kwargs={'expr': x['expr'], 'k': 25}
            ).invoke(x['question'])
        ).invoke({}),
    ).assign(
        context=lambda x: reranking(
            list({doc.metadata.get("pk"): doc for doc in (x['context_raw'])}.values()),
            x['question'], 15
        )
    ).assign(
        formatted_context=lambda x: format_docs(x['context'])
    )
    | RunnableLambda(
        lambda x: {
            "question": x['question'],
            "context": x['formatted_context'],  
        }
    )
    | text_prompt
    | llm_text
    | StrOutputParser()
)

In [25]:
llm_text = ChatOpenAI(model='o1', temperature=1)

answer = []

predict_prompt = PromptTemplate.from_template(
  '''You are future-predicting expert AI chatbot about financial.
  주어진 정보는 retrieved context들이야. 이 정보를 바탕으로 미래를 예측해줘.

  If one of the table or text says it doesn't know or it can't answer, don't mention with that.
  주어진 예측을 한 근거도 함께 자세히 설명해줘. 왜 그런 예측을 어떤 걸 근거로 내놓았는지 알려줘.
  Don't answer with the specific numbers.

  #Question:
  {question}

  #Context:
  {context}
  '''
)

predict_expression = 'issue_date >= "20241224" AND issue_date <="20250124"'
predict_chain = (
    RunnableParallel(
        question=itemgetter('question')
    ).assign(context=lambda x: RunnableLambda(
            lambda _: vectorstore_text.as_retriever(
                search_kwargs={'k': 20, 'expr':predict_expression}
            ).invoke(x['question'])
        ).invoke({}),
    ).assign(
        formatted_context=lambda x: format_docs(x['context'])
    )
    | RunnableLambda(
        lambda x: {
            "question": x['question'],
            "context": x['formatted_context'],  
        }
    )
    | predict_prompt
    | llm_text
)

In [26]:
text_chain_2 = (
    RunnableParallel(
        question=itemgetter('question')
    ).assign(expr = lambda x: get_query_date(x['question'])
    ).assign(milvus=lambda x: RunnableLambda(
            lambda _: vectorstore_text.as_retriever(
                search_kwargs={'k': 25}
            ).invoke(x['question'])
        ).invoke({}),
        bm25=lambda x: bm25_retriever_text.invoke(x['question'])
    ).assign(
        context=lambda x: reranking(
            list({doc.metadata.get("pk"): doc for doc in (x['milvus'] + x['bm25'])}.values()),
            x['question'], 20
        )
    ).assign(
        formatted_context=lambda x: format_docs(x['context'])
    )
    | RunnableLambda(
        lambda x: {
            "question": x['question'],
            "context": x['formatted_context'],  
        }
    )
    | text_prompt
    | llm_text
    | StrOutputParser()
)

In [27]:
table_prompt = PromptTemplate.from_template(
'''You are an assistant for question-answering tasks.
Use the following pieces of retrieved table to answer the question.
If you don't know the answer, just say that you don't know.
Answer in Korean. Answer in detail.

#Question:
{question}
#Context:
{context}

#Answer:'''
)

table_chain = (
    RunnableParallel(
        question=itemgetter('question')
    ).assign(expr = lambda x: get_query_date(x['question'])
    ).assign(milvus=lambda x: RunnableLambda(
            lambda _: vectorstore_table.as_retriever(
                search_kwargs={'expr': x['expr'], 'k': 10}
            ).invoke(x['question'])
        ).invoke({}),
        bm25_raw=lambda x: bm25_retriever_table.invoke(x['question'])
    ).assign(
        bm25_filtered=lambda x: [
            doc for doc in x["bm25_raw"]
            if not x["expr"] or (
                x["expr"].split("'")[1] <= doc.metadata.get("issue_date", "") <= x["expr"].split("'")[3]
            )
        ],
    ).assign(
    context=lambda x: list({
        doc.metadata.get("pk"): doc 
        for doc in (x['milvus'] + x['bm25_filtered'])
    }.values())
    ).assign(
        formatted_context=lambda x: format_docs(x['context'])
    )
    | RunnableLambda(
        lambda x: {
            "question": x['question'],
            "context": x['formatted_context'],  #
        }
    )
    | text_prompt
    | llm
    | StrOutputParser()
)

In [28]:
llm_general = ChatOpenAI(model='gpt-4o-mini', temperature=0)

general_prompt = PromptTemplate.from_template(
  '''You are question-answering AI chatbot about financial reports.
  주어진 정보는 retrieved context들이야. 이 정보를 바탕으로 질문에 대해 자세히 설명해줘.

  If one of the table or text says it doesn't know or it can't answer, don't mention with that.
  And some questions may not be answered simply with context, but rather require inference. In those cases, answer by inference.

  #Question:
  {question}

  #Context:
  {context}
  '''
)

date_chain = (
    RunnableParallel(
        question=itemgetter('question'),
        text=text_chain,
    )
    | kill_table
    | general_prompt
    | llm_general
)

In [29]:
general_chain = (
    RunnableParallel(
        question=itemgetter('question'),
        text=text_chain_2,
    )
    | kill_table
    | general_prompt
    | llm_general
)

In [30]:
from langchain_community.vectorstores import Milvus

embd = UpstageEmbeddings(
    model='solar-embedding-1-large-query',
)

vectorstore_raptor = Milvus(
    embedding_function=embd,
    connection_args={'uri':URI},
    index_params={'index_type': 'AUTOINDEX', 'metric_type': 'IP'},
    collection_name='raptor_v3'
)

In [31]:
metadata_field_info = [
  AttributeInfo(
    name='source',
    description='문서의 번호. 네 자리의 숫자와 "호"로 이루어져 있다. 현재 1090호부터 1120호까지 존재한다.',
    type='string',
  ),
]

prompt_query = get_query_constructor_prompt(
  'summary of weekly financial report about bonds',
  metadata_field_info
)

output_parser = StructuredQueryOutputParser.from_components()

query_llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)
query_constructor = prompt_query | query_llm | output_parser

In [32]:
from langchain.retrievers.self_query.milvus import MilvusTranslator

prompt_raptor = PromptTemplate.from_template(
'''You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Answer in Korean. Answer in detail.
If the context mentions an unrelated date, do not mention that part.
Summarize and organize your answers based on the various issues that apply to the period.

#Question:
{question}
#Context:
{context}

#Answer:'''
)

retriever_raptor = SelfQueryRetriever(
  query_constructor=query_constructor,
  vectorstore=vectorstore_raptor,
  structured_query_translator=MilvusTranslator(),
  search_kwargs={'k': 10}
)
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

raptor_chain = (
    RunnableParallel(
        question=itemgetter('question')
    ).assign(expr = lambda x: get_query_date(x['question'])
    ).assign(context=lambda x: retriever_raptor.invoke(x['question']))
    | RunnableLambda(
        lambda x: {
            "question": x['question'],
            "context": x['context'],
        }
    )
    | prompt_raptor
    | llm
)

In [33]:
raptor_date_chain = (
    RunnableParallel(
        question=itemgetter('question')
    ).assign(expr = lambda x: get_query_date(x['question'])
    ).assign(context=lambda x: RunnableLambda(
            lambda _: vectorstore_raptor.as_retriever(
                search_kwargs={'expr': x['expr'], 'k': 10}
            ).invoke(x['question'])
        ).invoke({})
    )
    | RunnableLambda(
        lambda x: {
            "question": x['question'],
            "context": x['context'],
        }
    )
    | prompt_raptor
    | llm
)

In [34]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

prompt_routing_2 = PromptTemplate.from_template(
  '''You are an expert at routing a user question to the appropriate data source.

If the user is asking for a brief summary, route it to the '요약' datasource.
If the user is asking for more detailed or general information, route it to the '일반' datasource.
If the user is asking for some prediction, route it to the '예측' datasource.

Just answer with one word of datasource.

Today is January 25th, 2025. Only classify as predictions asking about things after today.

  <question>
  {question}
  </question>

  datasource:'''
)

chain_routing_2 = (
  {'question': RunnablePassthrough()}
  | prompt_routing_2
  | ChatOpenAI(model='gpt-4o-mini')
  | StrOutputParser()
)

prompt_routing = PromptTemplate.from_template(
  '''주어진 사용자 질문을 `날짜`, `호수`, `일반` 중 하나로 분류하세요. 한 단어 이상으로 응답하지 마세요.
  If user question has the expression about date, route it to the '날짜' datasource.

  <question>
  {question}
  </question>

  Classification:'''
)

chain_routing = (
  {'question': RunnablePassthrough()}
  | prompt_routing
  | ChatOpenAI(model='o1')
  | StrOutputParser()
)

In [35]:
def route_2(info):
  if '요약' in info['topic'].lower():
    print('raptor_date_chain')
    return raptor_date_chain
  elif '예측' in info['topic'].lower():
    print('predict_chain')
    return predict_chain
  else:
    print('date_chain')
    return date_chain

def route(info):
  if '날짜' in info['topic'].lower():
    info['topic'] = chain_routing_2.invoke(info['question'])
    return route_2(info)
  elif '호수' in info['topic'].lower():
    print('raptor_chain')
    return raptor_chain
  else:
    print('general_chain')
    return general_chain


full_chain = (
  {'topic': chain_routing, 'question': itemgetter('question')}
  | RunnableLambda(
    route
  )
  | StrOutputParser()
)

In [36]:
filepath = './chunked_jsonl/image_v2.jsonl'

splitted_doc_image = []
with open(filepath, 'r', encoding='utf-8') as file:
    for line in file:
        if line.startswith('\n('):
            continue
        data = json.loads(line)

        doc = Document(
            page_content=data['page_content'],
            metadata=data['metadata']
        )
        splitted_doc_image.append(doc)

In [37]:
URI = 'http://127.0.0.1:19530'

vectorstore_image = Milvus(
    embedding_function=embeddings,
    connection_args={'uri':URI},
    index_params={'index_type': 'AUTOINDEX', 'metric_type': 'IP'},
    collection_name='image_v4'
)

In [38]:
retriever_image = vectorstore_image.as_retriever(search_kwargs={'k': 3})

In [39]:
query_retrieval_relevant = GroundednessChecker(
  llm=ChatOpenAI(model='gpt-4o-mini', temperature=0), target='question-retrieval'
).create()

In [40]:
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import rc


def ask(question):
    expr = get_query_date(question)
    answer = full_chain.invoke({'question': question})
    print(answer)
    rc('font', family='Malgun Gothic')
    plt.rcParams['axes.unicode_minus'] = False
    context = retriever_image.invoke(question, expr=expr)
    for i in context:
        rar = query_retrieval_relevant.invoke({'context': i, 'question': question})
        if rar.score=='yes':
            plt.title('참고 자료')
            image_path = i.metadata['image'].replace('raw_pdf_copy3', 'parsed_pdf')
            img = Image.open(image_path)
            plt.imshow(img)
            plt.axis('off')
            plt.show()

In [41]:
ask('2주 전 은행채 발행액은?')

date_chain
2주 전 은행채 발행액은 총 3,000백만원입니다. 이 금액은 2025년 1월 10일 기준으로 발행된 여러 은행채의 발행액을 합산한 결과입니다. 구체적으로는 국민은행, 산금, 우리은행, 농업금융채권, 부산은행 등 다양한 은행에서 발행된 채권들이 포함되어 있습니다. 각 은행채의 발행액은 다음과 같습니다:

- 국민은행4501이표일(03)1-06: 2,000백만원
- 산금25신이0106-0106-1: 1,400백만원
- 우리은행29-01-할인01-갑-06: 3,000백만원
- 농업금융채권(은행)2025-01이1Y-A: 700백만원
- 부산은행2025-01이1A-09: 1,000백만원
- 산금25신이0103-0109-1: 3,000백만원
- 산금25신이0200-0109-2: 7,000백만원
- 산금25신이0109-0110-1: 5,000백만원
- 한국수출입금융2501라-할인-304: 3,000백만원

이 중에서 2주 전 발행된 은행채의 총합이 3,000백만원이라는 점이 중요합니다.


In [42]:
ask('2주 전 은행채 발행액은?')

date_chain
2주 전 은행채 발행액은 총 3,000백만원입니다. 이 금액은 2025년 1월 10일 기준으로 발행된 여러 은행채의 발행액을 합산한 결과입니다. 구체적으로는 국민은행, 산금, 우리은행, 농업금융채권, 부산은행 등 다양한 은행에서 발행된 채권들이 포함되어 있습니다. 각 은행채의 발행액은 다음과 같습니다:

- 국민은행4501이표일(03)1-06: 2,000백만원
- 산금25신이0106-0106-1: 1,400백만원
- 우리은행29-01-할인01-갑-06: 3,000백만원
- 농업금융채권(은행)2025-01이1Y-A: 700백만원
- 부산은행2025-01이1A-09: 1,000백만원
- 산금25신이0103-0109-1: 3,000백만원
- 산금25신이0200-0109-2: 7,000백만원
- 산금25신이0109-0110-1: 5,000백만원
- 한국수출입금융2501라-할인-304: 3,000백만원

이들 중에서 2주 전 발행된 은행채의 총합이 3,000백만원이라는 점이 중요합니다.


In [43]:
ask('2주 전 은행채 발행액은?')

date_chain
2주 전 은행채 발행액은 총 15,100백만원입니다. 이 발행액은 여러 종류의 은행채로 구성되어 있으며, 구체적인 발행 내역은 다음과 같습니다:

- 국민은행4501이표일(03)1-06: 2,000백만원
- 산금25신이0106-0106-1: 1,400백만원
- 우리은행29-01-할인01-갑-06: 3,000백만원
- 농업금융채권(은행)2025-01이1Y-A: 700백만원
- 부산은행2025-01이1A-09: 1,000백만원
- 산금25신이0103-0109-1: 3,000백만원
- 산금25신이0200-0109-2: 7,000백만원
- 산금25신이0109-0110-1: 5,000백만원
- 한국수출입금융2501라-할인-304: 3,000백만원

이러한 발행액은 은행채 시장의 유동성과 자금 조달 상황을 반영하고 있습니다.


In [44]:
ask('2주 전 은행채 발행액은?')

date_chain
2주 전 은행채 발행액은 총 3,000백만원입니다. 이 금액은 2025년 1월 10일에 발행된 은행채의 발행액을 기준으로 하며, 여러 은행에서 발행된 다양한 채권들이 포함되어 있습니다. 각 은행별 발행액은 다음과 같습니다:

1. 국민은행4501이표일(03)1-06: 2,000백만원
2. 산금25신이0106-0106-1: 1,400백만원
3. 우리은행29-01-할인01-갑-06: 3,000백만원
4. 농업금융채권(은행)2025-01이1Y-A: 700백만원
5. 부산은행2025-01이1A-09: 1,000백만원
6. 산금25신이0103-0109-1: 3,000백만원
7. 산금25신이0200-0109-2: 7,000백만원
8. 산금25신이0109-0110-1: 5,000백만원
9. 한국수출입금융2501라-할인-304: 3,000백만원

이들 채권의 발행액을 모두 합산한 결과, 2주 전의 총 발행액이 3,000백만원으로 확인됩니다.
